In [40]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from __future__ import print_function, division
from scipy import stats
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from Utility.IO import *
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import cross_val_score
from sklearn.learning_curve import validation_curve
import math

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [94]:
def clean_and_prepare():
    # Load csv file into dataframe
    sc96 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED1996_97_PP.csv")
    sc97 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED1997_98_PP.csv")
    sc98 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED1998_99_PP.csv")
    sc99 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED1999_00_PP.csv")
    sc00 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2000_01_PP.csv")
    sc01 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2001_02_PP.csv")
    sc02 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2002_03_PP.csv")
    sc03 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2003_04_PP.csv")
    sc04 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2004_05_PP.csv")
    sc05 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2005_06_PP.csv")
    sc06 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2006_07_PP.csv")
    sc07 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2007_08_PP.csv")
    sc08 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2008_09_PP.csv")
    sc09 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2009_10_PP.csv")
    sc10 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2010_11_PP.csv")
    sc11 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2011_12_PP.csv")
    sc12 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2012_13_PP.csv")
    sc13 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2013_14_PP.csv")
    sc14 = pd.read_csv("../data/CollegeScorecard_Raw_Data/MERGED2014_15_PP.csv")
    
    # Load Dictionary
    dic_file=pd.ExcelFile("../data/CollegeScorecardDataDictionary.xlsx")
    dic_init = dic_file.parse("data_dictionary")
    dic_init.head()
    dic_init.columns
    
    # Clean Dictionary
    dic=dic_init.ix[dic_init['VARIABLE NAME'].notnull()]
    
    sc_init=[sc96,sc97,sc98,sc99,sc00,sc01,sc02,sc03,sc04,sc05,sc06,sc07,sc08,sc09,sc10,sc11,sc12,sc13,sc14]
    
    # Add year variable
    for i in range(len(sc_init)):
        sc_init[i]['Year']=i+1996
    
    # Merge dataset with valid repayment information (2007-2014)
    sc_merge=pd.concat(sc_init[11:19])
    
    # Pick x and y variables
    scx_merge=sc_merge.loc[:,dic['VARIABLE NAME'][dic["dev-category"]!="repayment"]]
    scy_merge=sc_merge.loc[:,['RPY_1YR_RT','RPY_3YR_RT','RPY_5YR_RT', 'RPY_7YR_RT'] ]
    
    # Combine x and y variables
    sc=pd.concat([scy_merge,scx_merge,sc_merge['Year']],axis=1)
    
    # Drop empty row and column
    sc_drop=sc.dropna(how="all",axis=1)
    sc_drop=sc_drop.dropna(how="all",axis=0)
    
    # Save it to local file
    sc_drop.to_csv('../data/sc_final.csv')
    
    return sc_drop

In [95]:
# Example
sc=clean_and_prepare()
# after creating .csv file, we can obtain data from file directly
sc= pd.read_csv("../data/sc_final.csv",encoding="ISO-8859-1")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2802: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2802: DtypeWarning: Columns (1,6,442,451,598,607,611,620,624,633,789,802,806,815,893,906,971,984,997,1153,1166,1407,1408,1411,1425,1431,1432,1433,1437,1438,1439,1440,1445,1446,1447,1451,1452,1453,1454,1459,1460,1461,1465,1466,1467,1468,1473,1474,1475,1479,1480,1481,1482,1487,1488,1489,1501,1502,1537,1538,1539,1540,1541,1542,1603,1606,1609,1610,1611,1613,1614,1615,1616,1688,1689,1690,1691,1692,1729) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2802: DtypeWarning: Columns (1,1729) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.r

In [37]:
def get_rpyrt_by_class(y=1):
    index={'1':1,'3':2,'5':3,'7':4}
    sc= pd.read_csv("../data/sc_final.csv",encoding="ISO-8859-1")
    yname='RPY_'+ str(y) +'YR_RT'
    sc_sub=pd.concat([sc.iloc[:,[index[str(y)]]],sc.drop(sc.columns[range(0,14)], axis=1)],axis=1)
    sc_sub=sc_sub.convert_objects(convert_numeric=True)
    sc_sub=sc_sub.dropna(subset=[yname])
    pro=0.3
    sc_sub=sc_sub.dropna(axis=0,how='all')
    sc_sub=sc_sub.dropna(axis=1,thresh=np.ceil(sc_sub.shape[1]*(1-pro)))
    return sc_sub

In [20]:
def get_corplot_by_class(y=1,thre=200):
    yname='RPY_'+ str(y) +'YR_RT'
    yname=math.log(1/(1-yname))
    sc_rpt=get_rpyrt_by_class(y)
    cor=sc_rpt.corr()
    corr=abs(cor).sort_values([yname],ascending=0)[yname]
    sc_cor=cor.loc[corr.index[0:thre],corr.index[0:thre]]
    fig, ax = plt.subplots(figsize=(10,7)) 
    sns.heatmap(abs(sc_cor),ax=ax, xticklabels=sc_cor.columns,yticklabels=sc_cor.columns)
    return cor.loc[corr.index[0:thre],corr.index[0:thre]]

In [19]:
sc_rpt.head()

,RPY_1YR_RT,Unnamed: 0,RPY_1YR_RT,SCH_DEG,MAIN,NUMBRANCH,PREDDEG,HIGHDEG,CONTROL,ST_FIPS,...,C100_L4,D100_L4,TRANS_4,DTRANS_4,TRANS_L4,DTRANS_L4,ICLEVEL,UGDS_MEN,UGDS_WOMEN,Year
0,0.477044,0,0.477044,3.0,1,1,3,4,1.0,1.0,...,NaN,NaN,0.000000,1038.0,NaN,NaN,1.0,0.4709,0.5291,2007
1,0.608185,1,0.608185,3.0,1,1,3,4,1.0,1.0,...,NaN,NaN,0.234477,1224.0,NaN,NaN,1.0,0.3956,0.6044,2007
2,0.657534,2,0.657534,3.0,1,1,3,4,2.0,1.0,...,NaN,NaN,0.600000,5.0,NaN,NaN,1.0,0.4752,0.5248,2007
3,0.657205,3,0.657205,3.0,1,1,3,4,1.0,1.0,...,NaN,NaN,0.331081,592.0,NaN,NaN,1.0,0.5169,0.4831,2007
4,0.432048,4,0.432048,3.0,1,1,3,4,1.0,1.0,...,NaN,NaN,0.000000,1263.0,NaN,NaN,1.0,0.4026,0.5974,2007


In [23]:
cat_list0=dic.category.unique()
cat_list=np.delete(cat_list0,[0,8])
cat_list

array(['school', 'admissions', 'academics', 'student', 'cost', 'aid',
       'completion', 'earnings'], dtype=object)

In [38]:
rpyrt0=get_rpyrt_by_class(1)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2802: DtypeWarning: Columns (1,2,3,4,6,11,12,13,14,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,6

In [41]:
rpyrt=rpyrt0
y=rpyrt.RPY_1YR_RT
rpyrt.RPY_1YR_RT=np.log(y/(1-y))

In [43]:
screen=prescreening_by_class(rpyrt,y=1,nobs=500)

In [54]:
X=rpyrt[screen].fillna(X.mean())
print (X.shape)
X=X/X.std()
print (X.shape)

(35027, 500)
(35027, 500)


In [ ]:
dic=dic.loc[dic['name'].isin(screen),:]

In [79]:
X.head()

,INC_PCT_H1,PELL_EVER,INC_PCT_LO,FAMINC,DEP_INC_PCT_H1,DEP_INC_AVG,MD_FAMINC,DEP_INC_PCT_LO,DEP_INC_PCT_H2,FEMALE_ENRL_ORIG_YR2_RT,...,PELL_COMP_4YR_TRANS_YR2_RT,MD_INC_UNKN_4YR_TRANS_YR8_RT,RET_PT4,MD_INC_COMP_ORIG_YR8_RT,DEP_WDRAW_ORIG_YR4_RT,NOLOAN_ENRL_4YR_TRANS_YR2_RT,PCIP14,LOAN_WDRAW_4YR_TRANS_YR4_RT,HI_INC_COMP_ORIG_YR8_RT,NOPELL_WDRAW_ORIG_YR6_RT
0,0.186241,1.626310,2.286272,1.701738,2.656822,1.812873,1.394737,1.178066,2.585756,1.403729,...,3.967138,6.039355,0.000000,1.720159,0.668542,2.550328,1.656943,0.518370,1.505387,0.684427
1,0.304054,1.153219,1.861945,2.242514,0.707591,2.703092,1.776965,0.776498,0.606445,1.317823,...,3.967138,6.039355,3.329313,2.098653,0.563798,0.419368,0.931189,2.581735,2.434203,0.797274
2,2.573762,1.447210,2.229139,1.601689,2.656822,2.095437,1.433666,3.660942,2.585756,0.639679,...,3.967138,6.039355,3.295137,3.651300,2.639820,2.550328,0.000000,2.581735,4.043541,1.666548
3,0.347913,1.251283,1.766677,2.395953,0.839262,2.932436,1.982026,0.687760,0.665897,1.179185,...,3.967138,6.039355,4.016380,1.606573,0.463674,2.550328,3.928505,0.349794,1.923550,0.674947
4,0.126675,1.693114,2.613336,1.347773,2.656822,1.433913,1.145989,1.380354,2.585756,1.212775,...,3.967138,6.039355,4.795522,1.137419,0.714825,2.550328,0.000000,0.635736,2.014562,0.811908


In [85]:
def rms_error(model, X, y):
    y_pred = model.predict(X)
    return np.sqrt(np.mean((y - y_pred) ** 2))


#pca = PCA(n_components=3)
var=0.8
X_trans=None
for cat in cat_list:
    variables=dic.loc[dic.category==cat,'name']
    XX=X.loc[:,np.array(variables)].fillna(0)
    pca=PCA(var)
    XX_trans=pca.fit_transform(XX)
    XX_trans=pd.DataFrame(XX_trans)
    X_trans=pd.concat([X_trans,XX_trans],axis=1)    
X_trans.shape

(35027, 64)

In [64]:
X_trans.head()

,0,1,2,3,0,0,1,2,3,4,...,25,26,27,28,29,30,31,0,1,2
0,2.575635,-0.726081,-0.744586,0.745023,2.582324,4.229074,-1.263753,-0.876405,2.523195,-0.911053,...,-0.479132,0.382393,0.393336,-0.844711,-0.290354,-0.537344,-0.942831,5.339613,0.484399,-0.727331
1,2.837385,-0.859578,-0.841028,0.970894,7.769753,7.401047,-2.462273,-1.685855,1.129596,1.663057,...,0.629662,1.849623,-0.740628,-0.525452,1.261704,0.308248,-1.156335,8.293232,1.754045,-0.886684
2,2.082918,-0.116335,-0.754694,-0.541850,-2.204871,-1.616608,-0.446363,1.813560,-0.801777,-1.130400,...,-0.439659,0.814027,0.464257,0.568661,-1.037171,-0.388159,0.518752,-0.294772,-1.598262,-1.246646
3,2.956247,-0.645707,-0.139537,0.536151,7.876664,5.829550,-1.434239,-1.699369,2.227802,-1.360302,...,-0.456996,0.537351,1.164540,0.688101,-0.092114,-0.167669,-0.548623,9.442851,2.397093,-1.165805
4,2.809084,-0.592255,-0.143973,0.451449,4.656560,4.595616,-1.881837,2.175090,0.409098,0.028869,...,0.003415,-1.282238,-0.076419,1.987584,-0.727227,0.922429,0.192714,4.236708,-0.140284,-0.510136


In [65]:
XX_trans.head()

,0,1,2
0,5.339613,0.484399,-0.727331
1,8.293232,1.754045,-0.886684
2,-0.294772,-1.598262,-1.246646
3,9.442851,2.397093,-1.165805
4,4.236708,-0.140284,-0.510136


In [78]:
#validation_curve(LinearRegression(), X_trans, y, cv=10,scoring=rms_error)
cv = cross_val_score(LinearRegression(), X_trans, y, cv=10)
print (cv)
cv.mean()

[ 0.58967657  0.55136778  0.73203124  0.73674018  0.71200744  0.78969919
  0.77577268  0.80310757  0.79335945  0.74189215]


0.72256542601914941

In [ ]:
sd=pd.DataFrame(np.std(sc_sub[x.index]))
new = sd[sd[0] >0.15]  
new.head()

In [ ]:
data=sc_sub[new.index]
y_var=data.RPY_1YR_RT
x_var=data.drop('RPY_1YR_RT', 1)
x_var=x_var.fillna(x_var.mean())
x_var=(x_var-x_var.mean())/x_var.std()
y_var=y_var.fillna(y_var.mean())
y_var=np.log(y_var/(1-y_var))
y_var

In [ ]:
model = LinearRegression()

In [ ]:
from sklearn.cross_validation import cross_val_score
cv = cross_val_score(model, x_var, y_var, cv=10)
cv.mean()

In [ ]:
cross_val_score(model, x_var, y_var, cv=10)

In [ ]:
np.exp(cv.mean())

In [ ]:
cross_val_score(model, x_var, y_var, cv=10)

In [ ]:
lm1 = smf.ols(formula='RPY_1YR_RT ~ PELL_EVER + INC_PCT_LO +  FAMINC + MD_FAMINC + DEP_INC_AVG + DEP_INC_PCT_LO + FEMALE_ENRL_ORIG_YR2_RT + MALE_ENRL_ORIG_YR2_RT + PELL_ENRL_ORIG_YR2_RT + NOT1STGEN_ENRL_ORIG_YR2_RT + NOPELL_ENRL_ORIG_YR2_RT + NOLOAN_ENRL_ORIG_YR2_RT + ENRL_ORIG_YR2_RT', data=sc_sub).fit()
lm1.summary()